In [7]:
import os

# Replace 'your/directory/path' with the path to the directory you want to set as the current working directory
os.chdir('/teamspace/studios/this_studio/prithvi-pytorch')

# To verify that the current working directory has been changed, you can use:
print(os.getcwd())

/teamspace/studios/this_studio/prithvi-pytorch


In [8]:
import lightning
from torchgeo.trainers import SemanticSegmentationTask

from prithvi_pytorch import PrithviUnet
from prithvi_pytorch.datasets import HLSBurnScarsDataModule

CKPT_PATH = "weights/Prithvi_100M.pt"
CFG_PATH = "weights/Prithvi_100M_config.yaml"

In [9]:
class PrithviSegmentationTask(SemanticSegmentationTask):
    def configure_models(self):
        self.model = PrithviUnet(
            num_classes=self.hparams["num_classes"],
            cfg_path=CFG_PATH,
            ckpt_path=CKPT_PATH,
            in_chans=self.hparams["in_channels"],
            img_size=512,
            n=[2, 5, 8, 11],
            norm=False,
            decoder_channels=[256, 128, 64, 32],
            freeze_encoder=False,
        )

In [10]:
module = PrithviSegmentationTask(
    in_channels=6, num_classes=2, loss="focal", lr=1e-3, patience=10, ignore_index=0
)
datamodule = HLSBurnScarsDataModule(
    root="data/hls_burn_scars",
    batch_size=4,
    num_workers=8,
)

In [11]:
trainer = lightning.Trainer(
    accelerator="gpu", 
    logger=True, 
    max_epochs=20, 
    precision="16-mixed"
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model=module, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | FocalLoss        | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | test_metrics  | MetricCollection | 0     
4 | model         | PrithviUnet      | 119 M 
---------------------------------------------------
118 M     Trainable params
1.3 M     Non-trainable params
119 M     Total params
479.962   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [13]:
trainer.test(datamodule=datamodule)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /teamspace/studios/this_studio/prithvi-pytorch/lightning_logs/version_3/checkpoints/epoch=19-step=2700.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /teamspace/studios/this_studio/prithvi-pytorch/lightning_logs/version_3/checkpoints/epoch=19-step=2700.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric         ┃        DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   test_MulticlassAccuracy   │             1.0             │
│ test_MulticlassJaccardIndex │             1.0             │
│          test_loss          │    3.453059171931727e-09    │
└─────────────────────────────┴─────────────────────────────┘

[{'test_loss': 3.453059171931727e-09,
  'test_MulticlassAccuracy': 1.0,
  'test_MulticlassJaccardIndex': 1.0}]

```
[{'test_loss': 3.453059171931727e-09,
  'test_MulticlassAccuracy': 1.0,
  'test_MulticlassJaccardIndex': 1.0}]
```